In [17]:
import plotly.express as px
import numpy as np
import pandas as pd
import numpy as np
from faker import Faker
from dash import dcc
import dash
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium

In [18]:
#Create 

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 500

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7,9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17,22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

#Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))


aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

practice_df.head()

# Define buffer distances
buffer_distances = {
    'outside': 200,  # ~200 meters in degrees
    'inside': 150,   # ~150 meters in degrees
    'difficult': 125  # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Convert DataFrame to GeoDataFrame
practice_gdf = gpd.GeoDataFrame(
    practice_df,
    geometry=[Point(xy) for xy in zip(practice_df['Longitude'], practice_df['Latitude'])],
    crs="EPSG:4326"  # WGS84 coordinate system
)

def make_colors(selected_day, selected_hour):
    colors = []
    for index, row in practice_df.iterrows():
        day = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if any(d in selected_day for d in day):
            if opening_hour <= selected_hour <= closing_hour:
                color = 'green'
            else:
                color = 'red'
        else:
            color = 'red'
        colors.append(color)
    return colors

def generate_folium_map(selected_day, selected_hour):
    colors = make_colors(selected_day, selected_hour)
    m = folium.Map(location=[50.5, 4.3517], zoom_start=6)
    for idx, row in practice_gdf.iterrows():
        folium.Circle(
            location=(row['Latitude'], row['Longitude']),
            radius=row['buffer_distance'],
            color=colors[idx],
            fill=True,
            fill_opacity=row['opacity']
        ).add_to(m)
    return m.get_root().render()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id='day-radioitems',
        options=[{'label': day, 'value': day} for day in daysofweek_categories],
        value='Monday',
        inline=True
    ),
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    html.Iframe(
        id='folium-map',
        srcDoc=generate_folium_map('Monday', 12),
        width='100%',
        height='600px'
    )
])

@app.callback(
    Output('folium-map', 'srcDoc'),
    [Input('day-radioitems', 'value'), Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    return generate_folium_map(selected_day, selected_hour)

if __name__ == '__main__':
    app.run_server(debug=True)

In [19]:
practice_df.head()

,daysofweek,opening_hour,closing_hour,opacity,Latitude,Longitude,aed_coordinates,aed_placement,buffer_distance
0,"[Wednesday, Monday]",7,20,0.803778,50.076697,5.014547,"(50.07669720720639, 5.014547175698947)",difficult,125
1,"[Sunday, Thursday, Saturday, Wednesday]",8,21,0.224610,50.430899,5.225366,"(50.4308989784668, 5.225365905371102)",outside,200
2,"[Saturday, Friday, Sunday, Thursday, Wednesday]",8,21,0.324472,50.492911,3.511150,"(50.49291103358582, 3.5111496376888884)",difficult,125
3,"[Monday, Saturday, Thursday, Sunday, Wednesday...",7,19,0.408911,50.932337,4.607664,"(50.93233748382197, 4.6076639777131865)",difficult,125
4,"[Monday, Saturday, Thursday, Friday, Tuesday]",8,18,0.318911,50.653501,4.308329,"(50.653500691835404, 4.308329460285654)",inside,150
